# Imports

In [1]:
import numpy as np
import pandas as pd
import random as rand

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D
from keras.models import Sequential

from sklearn.model_selection import train_test_split

2023-07-20 09:03:52.913173: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 09:03:52.974130: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 09:03:52.975856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 09:03:54.044419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data importing and pre-processing

In [2]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 64, 64)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [14]:
def generate_generator_input(no_of_samples):
    input = np.random.randn(64*no_of_samples)

    input = input.reshape(no_of_samples,8,8)

    return input

In [15]:
def make_generator():
    model = Sequential()
    model.add(Conv2DTranspose(64, (8,8), strides=(2,2), padding='same'))
    model.add(Activation("relu"))
    model.add(Reshape((64,16,16)))
    
    model.add(Conv2DTranspose(64, (16,16), strides=(2,2), padding='same'))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(64, (64,64), strides=(2,2), padding='same'))
    model.add(Activation("relu"))
    
    return model
    
    

# Discriminator Model

In [16]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(1,64,64)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    
    
    # output layer
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    model.add(Activation("sigmoid"))
    

# Training

First, create the models from the functions

In [17]:
gen_model = make_generator()
disc_model = make_discriminator()


Next, create fake training data, and put it together with the real data.

In [18]:
gen_input = generate_generator_input(60)

print(gen_input.shape)

x_fake = gen_model.predict(gen_input)
y_fake = ones((60, 1))


    
x_fake_train, x_fake_test = train_test_split(x_fake, test_size=0.2)
y_fake_train, y_fake_test = train_test_split(y_fake, test_size=0.2)

x_train = np.concatenate(x_fake_train, x_real_train)
y_train = np.concatenate(y_fake_train, y_real_train)
x_train = np.concatenate(x_fake_test, x_real_test)
y_train = np.concatenate(y_fake_test, y_real_test)

(60, 8, 8)


ValueError: in user code:

    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 253, in assert_input_